# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Построим модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке самостоятельно.

## Открываем и изучаем файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
#загружаем библиотеки необходимые для дальнейшей работы

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.shape)
display(df.head(5))
#считываем датасет и выводим информацию по нему

(3214, 5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбиваем данные на выборки

In [3]:

df_train, df_valid = train_test_split(df, test_size=.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid, test_size=.5, random_state=12345)
display(f'Размез тренировочной выборки (строк/столбцов):{df_train.shape}')
display(f'Размез валидационной выборки (строк/столбцов):{df_valid.shape}')
display(f'Размез тестовой выборки (строк/столбцов):{df_test.shape}')
# < разделяем данные на обучающую, валидационную и тестовую выборки и проверяем их размеры>

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
# < создаем переменные для признаков и целевого признака >



'Размез тренировочной выборки (строк/столбцов):(1928, 5)'

'Размез валидационной выборки (строк/столбцов):(643, 5)'

'Размез тестовой выборки (строк/столбцов):(643, 5)'

### Вывод
Мы разделили данные на три выборки: тренировочную, валидационную и тестовую. Так же мы создали переменные для признаков и целевого признака.

## Исследуем модели

In [4]:
model = DecisionTreeClassifier()
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
display(f'Accuray стандартной модели "дерево решений" на валидационной выборке: {result}',)

'Accuray стандартной модели "дерево решений" на валидационной выборке: 0.7293934681181959'

In [5]:
best_model_dtc = None
best_result_dtc = 0
for depth in range(1,20):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    if result > best_result_dtc:
        best_model_dtc = model
        best_result_dtc = result
print(f'Accuracy наилучшей модели "дерево решений" на валидационной выборке: {best_result_dtc}')
print()
print(f'Параметры лучшей модели: {best_model_dtc}')

Accuracy наилучшей модели "дерево решений" на валидационной выборке: 0.7293934681181959

Параметры лучшей модели: DecisionTreeClassifier(max_depth=1, random_state=12345)


In [6]:
model = RandomForestClassifier()
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
display(f'Accuray модели "случайный лес" на валидационной выборке: {result}')

'Accuray модели "случайный лес" на валидационной выборке: 0.7869362363919129'

In [7]:
best_model_rfc = None
best_result_rfc = 0
for est in range(1, 11):
    for dep in range(1, 11):
        for n_samples in range(1, 11):
            model = RandomForestClassifier(random_state=12345,
                                       max_depth=dep,
                                       n_estimators=est, 
                                       min_samples_leaf = n_samples) # обучите модель с заданным количеством деревьев
            model.fit(features_train, target_train) # обучите модель на тренировочной выборке
            result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
            if result > best_result_rfc:
                best_model_rfc = model
                best_result_rfc = result#  сохраните наилучшее значение метрики accuracy на валидационных данных

print(f'Accuracy наилучшей модели на валидационной выборке: {best_result_rfc}')
print()
print(f'Параметры лучшей модели: {best_model_rfc}')

Accuracy наилучшей модели на валидационной выборке: 0.807153965785381

Параметры лучшей модели: RandomForestClassifier(max_depth=7, min_samples_leaf=6, n_estimators=10,
                       random_state=12345)


In [8]:
model_lr = LogisticRegression()
model_lr.fit(features_train, target_train)
result_lr = model.score(features_valid, target_valid)
display(f'Accuray модели "логистической регрессии" на валидационной выборке: {result_lr}')

'Accuray модели "логистической регрессии" на валидационной выборке: 0.7931570762052877'

### Вывод

Мы посторили 3 модели, для некоторых из них прошлись циклом меняя гиперпараметры, по итогу лучшей моделью по качеству оказалась `RandomForestClassifier`.

## Проверяем модель на тестовой выборке

In [9]:
model = RandomForestClassifier( max_depth=7, n_estimators=10, min_samples_leaf=6, random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print("Accuracy наилучшей модели на валидационной выборке:", best_result_rfc)
display(f'Accuray модели "случайный лес" на тестовой выборке: {result}')

Accuracy наилучшей модели на валидационной выборке: 0.807153965785381


'Accuray модели "случайный лес" на тестовой выборке: 0.7900466562986003'

### Вывод

Сравнив модели по найлучшей модели на валидационной выборке, мы видим, что разница в результатах не велика, 0.8 и 0.79 соответственно.

## Проверяем модели на адекватность

In [10]:
dc_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dc_model.fit(features_train, target_train)
result_dc = dc_model.score(features_valid, target_valid)
display(f'Accuracy наилучшей модели на валидационной выборке: {best_result_rfc}')
display(f'Accuray модели "случайный лес" на тестовой выборке: {result}')
display(f'Accuray модели "фиктивный классификатор" на валидацинной выборке: {result_dc}')

'Accuracy наилучшей модели на валидационной выборке: 0.807153965785381'

'Accuray модели "случайный лес" на тестовой выборке: 0.7900466562986003'

'Accuray модели "фиктивный классификатор" на валидацинной выборке: 0.7060653188180405'

### Вывод

Проверили модели на адекватность при помощи `DummyClassifier`. Судя по разности значений, наши модели вполне вменяемы. Подводя итоги, можно сказать, что наилучшей моделью оказалась модель :`RandomForestClassifier` ее значения `accuray` при определенных параметрах достигли 0.8.